In [ ]:
import serial
import time

def send_cmd(com_port, string1, display):
    # Open COM object
    ser = serial.Serial(
        port=com_port,
        baudrate=115200,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        bytesize=serial.EIGHTBITS,
        timeout=0.1
    )

    # Perform direct communication with device
    to_send = string1.encode() + b'\r\n'  # Add CR and LF as in MATLAB

    # Send command
    ser.write(to_send)
    time.sleep(0.5)  # Wait for response

    # Read response
    response = ser.read(ser.in_waiting or 1).decode('utf-8')
    num = len(response)

    if num == 0:
        print('timeout was reached.')
    else:
        # Display
        if display:
            print(f'String sent to Device: "{string1}"')
            print(f'Response from Device (in HEX): "{response.strip()}"')
            print(f'Number of Chars received: {num - 2} (+2 terminator chars)')
            print('-')

    # Close COM Stream
    ser.close()
    return response, num


def add_error_code(string):
    if len(string) % 2 == 0:
        print('Error: Input string length must be odd.')
        return -1
    else:
        temp = 0x100000000  # Equivalent to hex2dec('ffffffff') + 1
        string = string[1:]  # Remove the first character

        for i in range(0, len(string) - 1, 2):
            temp -= int(string[i:i+2], 16)

        temp_hex = '{:X}'.format(temp)
        complete_string = ':' + string + temp_hex[-2:]
        return complete_string

def home_z(com_port):
    # Switch Modbus 'z' on
    command = add_error_code(':01050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on
    command = add_error_code(':01050403FF00')
    send_cmd(com_port, command, True)

    # Home Device
    command = add_error_code(':0105040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command
    command = add_error_code(':0105040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':010390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F2" in response:
            print("Homing process completed successfully.")
            homing_completed = True
        else:
            print("Waiting for homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1



def get_position_z(com_port):
    # Send the command to get the Z position
    command = add_error_code(':010390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    # Adjusted index for Python string and multiplied by 2 since each hex digit represents half a byte
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    return position

def move_z(com_port, distance, speed, acceleration, timeout):
    # Validate input values
    if not (0 <= distance <= 150) or not (0 < speed <= 100) or not (acceleration >= 0):
        print("ERROR: Invalid distance, speed, or acceleration values.")
        return -1

    # Convert distance, speed, and acceleration to hexadecimal strings with proper formatting
    distance_hex = '{:08X}'.format(int(distance * 100))
    speed_hex = '{:08X}'.format(int(speed * 100))
    acceleration_hex = '{:08X}'.format(int(acceleration * 100))

    # Assuming the device expects the speed and acceleration to be split or formatted in a specific way
    # Adjust the slicing/indexing as per the device's protocol
    command_details = f"{distance_hex}{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000"
    command = f":01109900000912{command_details}"

    complete_command = add_error_code(command)  # Ensure this function is correctly appending the checksum


    print("Sending moveZ command:", complete_command)
    response, _ = send_cmd(com_port, complete_command, True)
    print("Response:", response)

    # Wait for movement completion here as per device protocol

    return get_position_z(com_port)



# Main sequence
com_port = 'COM3'  # Adjust as needed

# 1. Home Z
print("Homing Z stage...")
home_z(com_port)

# 2. Move Z
# The command you want to send, which is known to work
#command = ":0110990000091200000FA000000001000009C4001E00000000A0"

# Send the command directly using the send_cmd function
#print("Sending known good command to move Z stage...")
#response, num_chars_received = send_cmd(com_port, command, True)

# Print the response from the device
#print(f"Response from Device (in HEX): '{response}'")
#print(f"Number of Chars received: {num_chars_received - 2} (+2 terminator chars)")


pos = 40  # final position (mm)
vel = 25  # velocity (mm/s)
acc = 0.3  # acceleration (mm/s^2)
print(f"Moving Z stage to {pos}mm at {vel}mm/s with {acc}mm/s^2 acceleration.")
move_z(com_port, pos, vel, acc, 0)

# 3. Get Position Z
#position = get_position_z(com_port)
#print(f"Current Z position: {position}mm")

# 4. Home Z again
#print("Homing Z stage again...")
#home_z(com_port)

# Fully working implementation of moving and homing the z function

In [5]:
import serial
import time

def send_cmd(com_port, string1, display):
    ser = serial.Serial(port=com_port, baudrate=115200, parity=serial.PARITY_NONE,
                        stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=0.1)
    to_send = string1.encode() + b'\r\n'
    ser.write(to_send)
    time.sleep(0.5)  # Delay for response
    response = ser.read(ser.in_waiting or 1).decode('utf-8')
    num = len(response)
    if num == 0:
        print('Timeout was reached.')
    elif display:
        print(f'String sent to Device: "{string1}"')
        print(f'Response from Device (in HEX): "{response.strip()}"')
        print(f'Number of Chars received: {num - 2} (+2 terminator chars)')
    ser.close()
    return response, num

def add_error_code(string):
    if len(string) % 2 == 0:
        print('Error: Input string length must be odd.')
        return -1
    temp = 0x100000000  # Equivalent to hex2dec('ffffffff') + 1
    string = string[1:]  # Remove the first character
    for i in range(0, len(string) - 1, 2):
        temp -= int(string[i:i+2], 16)
    temp_hex = '{:X}'.format(temp)
    complete_string = ':' + string + temp_hex[-2:]
    return complete_string

def home_z(com_port):
    # Switch Modbus 'z' on
    command = add_error_code(':01050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on
    command = add_error_code(':01050403FF00')
    send_cmd(com_port, command, True)

    # Home Device
    command = add_error_code(':0105040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command
    command = add_error_code(':0105040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':010390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F2" in response:
            print("Homing process completed successfully.")
            homing_completed = True
        else:
            print("Waiting for homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1


def get_position_z(com_port):
    # Send the command to get the Z position
    command = add_error_code(':010390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    # Adjusted index for Python string and multiplied by 2 since each hex digit represents half a byte
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    return position

def format_hex(value, length):
    """Format the given value as a hexadecimal string of the specified length."""
    return f'{value:0{length}X}'

def calculate_checksum(command):
    """Calculate the two's complement checksum for the given command."""
    sum_bytes = sum(int(command[i:i+2], 16) for i in range(1, len(command), 2))  # Start from 1 to skip ':'
    checksum = (0x10000 - sum_bytes) & 0xFFFF
    return format_hex(checksum, 4)[-2:]

def move_z(distance, speed, acceleration):
    """Construct the move Z command based on distance, speed, and acceleration."""
    if distance < 0 or distance > 150 or speed <= 0 or speed > 100:
        print("ERROR: Invalid Position or Speed.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'01109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    print(f"Constructed command for sending: {complete_command_with_checksum}")
    return complete_command_with_checksum


# Main sequence
def main():
    com_port = 'COM3'  # Adjust to your COM port
    distance = 40  # Desired distance in mm
    speed = 25  # Desired speed in mm/s
    acceleration = 0.3  # Desired acceleration in mm/s^2
    timeout = 0.5  # Timeout for waiting after command sent

    # Home the Z stage first
    print("Homing Z stage...")
    home_z(com_port)  # Ensure home_z function is defined and working as expected

    # Generate the move Z command
    move_command = move_z(distance, speed, acceleration)
    if move_command:
        # Send the move Z command
        print(f"Moving Z stage to {distance}mm at {speed}mm/s with {acceleration}mm/s^2 acceleration.")
        send_cmd(com_port, move_command, True)

        # Wait for movement to complete or implement additional checks as needed
        time.sleep(timeout)  # Simple wait; adjust based on your system's needs

        # Optionally, retrieve and print the final position
        final_position = get_position_z(com_port)  # Ensure get_position_z is correctly implemented
        print(f"Final Z position: {final_position}mm")

if __name__ == "__main__":
    main()

Homing Z stage...
String sent to Device: ":01050427FF00D0"
Response from Device (in HEX): ":01050427FF00D0"
Number of Chars received: 15 (+2 terminator chars)
String sent to Device: ":01050403FF00F4"
Response from Device (in HEX): ":01050403FF00F4"
Number of Chars received: 15 (+2 terminator chars)
String sent to Device: ":0105040BFF00EC"
Response from Device (in HEX): ":0105040BFF00EC"
Number of Chars received: 15 (+2 terminator chars)
String sent to Device: ":0105040B0000EB"
Response from Device (in HEX): ":0105040B0000EB"
Number of Chars received: 15 (+2 terminator chars)
String sent to Device: ":01039005000166"
Response from Device (in HEX): ":01030270800A"
Number of Chars received: 13 (+2 terminator chars)
Waiting for homing to complete... Response:  :01030270800A

String sent to Device: ":01039005000166"
Response from Device (in HEX): ":0103027098F2"
Number of Chars received: 13 (+2 terminator chars)
Homing process completed successfully.
Constructed command for sending: :0110990

# Test move Z offline

In [ ]:
def format_hex(value, length):
    """Format the given value as a hexadecimal string of the specified length."""
    return f'{value:0{length}X}'

def calculate_checksum(command):
    """Calculate the two's complement checksum for the given command."""
    sum_bytes = sum(int(command[i:i+2], 16) for i in range(1, len(command), 2))  # Start from 1 to skip ':'
    checksum = (0x10000 - sum_bytes) & 0xFFFF
    return format_hex(checksum, 4)[-2:]

def move_z_python(distance, speed, acceleration):
    """Construct the move Z command based on distance, speed, and acceleration."""
    if distance < 0 or distance > 150 or speed <= 0 or speed > 100:
        print("ERROR: Invalid Position or Speed.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'01109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    print(f"Constructed command for sending: {complete_command_with_checksum}")
    return complete_command_with_checksum

# Example usage
distance = 40  # mm
speed = 25  # mm/s
acceleration = 0.3  # mm/s^2
move_z_python(distance, speed, acceleration)


String sent to Device: ":0110990000091200000FA000000001000009C4001E00000000A0"

# Code to move x axis

In [7]:
import serial
import time

def send_cmd(com_port, string1, display):
    ser = serial.Serial(port=com_port, baudrate=115200, parity=serial.PARITY_NONE,
                        stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=0.1)
    to_send = string1.encode() + b'\r\n'
    ser.write(to_send)
    time.sleep(0.5)  # Delay for response
    response = ser.read(ser.in_waiting or 1).decode('utf-8')
    num = len(response)
    if num == 0:
        print('Timeout was reached.')
    elif display:
        print(f'String sent to Device: "{string1}"')
        print(f'Response from Device (in HEX): "{response.strip()}"')
        print(f'Number of Chars received: {num - 2} (+2 terminator chars)')
    ser.close()
    return response, num

def add_error_code(string):
    if len(string) % 2 == 0:
        print('Error: Input string length must be odd.')
        return -1
    temp = 0x100000000  # Equivalent to hex2dec('ffffffff') + 1
    string = string[1:]  # Remove the first character
    for i in range(0, len(string) - 1, 2):
        temp -= int(string[i:i+2], 16)
    temp_hex = '{:X}'.format(temp)
    complete_string = ':' + string + temp_hex[-2:]
    return complete_string


def home_x(com_port):
    # Switch Modbus 'x' on
    command = add_error_code(':03050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':03050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0305040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0305040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':030390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F0" in response:
            print("Homing process for X axis completed successfully.")
            homing_completed = True
        else:
            print("Waiting for X axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1

def get_position_x(com_port):
    # Send the command to get the X position
    command = add_error_code(':030390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current X position: {position}mm")
    return position

def format_hex(value, length):
    """Format the given value as a hexadecimal string of the specified length."""
    return f'{value:0{length}X}'

def calculate_checksum(command):
    """Calculate the two's complement checksum for the given command."""
    sum_bytes = sum(int(command[i:i+2], 16) for i in range(1, len(command), 2))  # Start from 1 to skip ':'
    checksum = (0x10000 - sum_bytes) & 0xFFFF
    return format_hex(checksum, 4)[-2:]


# Function to move X axis
def move_x(distance, speed, acceleration):
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for X axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'03109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    print(f"Constructed command for sending: {complete_command_with_checksum}")
    return complete_command_with_checksum


def main():
    com_port = 'COM3'  # Adjust to your COM port
    distance = 20  # Desired distance in mm for X axis
    speed = 15  # Desired speed in mm/s for X axis
    acceleration = 0.5  # Desired acceleration in mm/s^2 for X axis
    #timeout = 0.5  # Timeout for waiting after command sent

    # Home the X stage first
    print("Homing X stage...")
    home_x(com_port)  # Ensure home_x function is defined and working as expected

    # Generate the move X command
    print(f"Moving X stage to {distance}mm at {speed}mm/s with {acceleration}mm/s^2 acceleration.")
    move_x(distance, speed, acceleration)  # Ensure move_x is correctly implemented

    # Wait for movement to complete or implement additional checks as needed
    #time.sleep(timeout)  # Simple wait; adjust based on your system's needs

    # Retrieve and print the final position of the X stage
    final_position_x = get_position_x(com_port)  # Ensure get_position_x is correctly implemented
    print(f"Final X position: {final_position_x}mm")

if __name__ == "__main__":
    main()




Homing X stage...
String sent to Device: ":03050427FF00CE"
Response from Device (in HEX): ":03050427FF00CE"
Number of Chars received: 15 (+2 terminator chars)
String sent to Device: ":03050403FF00F2"
Response from Device (in HEX): ":03050403FF00F2"
Number of Chars received: 15 (+2 terminator chars)
String sent to Device: ":0305040BFF00EA"
Response from Device (in HEX): ":0305040BFF00EA"
Number of Chars received: 15 (+2 terminator chars)
String sent to Device: ":0305040B0000E9"
Response from Device (in HEX): ":0305040B0000E9"
Number of Chars received: 15 (+2 terminator chars)
String sent to Device: ":03039005000164"
Response from Device (in HEX): ":030302708008"
Number of Chars received: 13 (+2 terminator chars)
Waiting for X axis homing to complete... Response:  :030302708008

String sent to Device: ":03039005000164"
Response from Device (in HEX): ":0303027098F0"
Number of Chars received: 13 (+2 terminator chars)
Homing process for X axis completed successfully.
Moving X stage to 20mm 

# Debug move x

In [4]:
def debug_move_x(distance, speed, acceleration):
    """Construct the move X command for debugging purposes."""
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for X axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal strings with proper formatting
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Assemble the command string for the X axis.
    command_details = f"{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000"
    command = f":02109900000912{command_details}"

    # Calculate and append the checksum to the command.
    complete_command_with_checksum = add_error_code(command[1:])  # Remove the leading ':' for checksum calculation

    if complete_command_with_checksum == -1:
        print("Error calculating checksum. Command construction failed.")
        return

    # Instead of sending, print the command for offline debugging
    print(f"Constructed command for sending (Debug): {complete_command_with_checksum}")
    return complete_command_with_checksum

# Example usage for debugging
distance = 20  # Desired distance in mm
speed = 15  # Desired speed in mm/s
acceleration = 0.5  # Desired acceleration in mm/s^2
debug_move_x(distance, speed, acceleration)

def move_x(distance, speed, acceleration):
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for X axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'03109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    print(f"Constructed command for sending: {complete_command_with_checksum}")
    return complete_command_with_checksum


Error: Input string length must be odd.
Error calculating checksum. Command construction failed.


String sent to Device: ":03109900000912000011940000000100001388001E00000000DA"

#move y

String sent to Device: ":02109900000912000011940000000100001388001E00000000DB"

movvvee pphi

String sent to Device: ":04109900000912000088B80000000100001388001E000000003E